In [63]:
import random
from datetime import datetime, timedelta
import networkx as nx
from pyvis.network import Network
from collections import deque
import community

### 예시 데이터 만들기

In [76]:
transactions = []
names = [
    '신소율', '홍수아', '홍지호', '신민지', '신도윤', '한민준', '안현우', '최하은', '권현준', '홍지은',
    '윤지원', '김태윤', '이주원', '박서준', '최예은', '강지호', '배수아', '정하늘', '오유진', '서민재',
    '장하준', '민서연', '김다은', '송지후', '이하린', '윤지우', '한지호', '최다은', '정민준', '서지우'
]

start_time = datetime(2024, 1, 1, 9, 0, 0)

for _ in range(100):
    sender = random.choice(names)
    receiver = random.choice(names)

    # 출금인과 송금인이 같으면 재선택
    while sender == receiver:
        receiver = random.choice(names)
    
    amount = random.randint(10000, 5000000)
    time_delta = timedelta(minutes=random.randint(0, 60*24*30))
    transaction_time = (start_time + time_delta).strftime('%m-%d %H:%M')

    transactions.append((sender, receiver, amount, transaction_time))

In [77]:
transactions

[('안현우', '민서연', 3185047, '01-24 13:16'),
 ('한민준', '윤지우', 1794250, '01-13 02:28'),
 ('최예은', '권현준', 4241976, '01-12 13:22'),
 ('홍수아', '민서연', 993110, '01-03 04:37'),
 ('민서연', '한지호', 3329964, '01-05 09:10'),
 ('이주원', '박서준', 4103683, '01-09 10:41'),
 ('이주원', '홍지은', 3851091, '01-03 08:19'),
 ('김태윤', '최다은', 4009586, '01-11 22:52'),
 ('오유진', '최다은', 1042404, '01-18 04:31'),
 ('최하은', '한지호', 2913770, '01-19 21:50'),
 ('최하은', '홍수아', 1682278, '01-16 16:35'),
 ('한민준', '윤지우', 1451364, '01-04 14:14'),
 ('신소율', '안현우', 3540749, '01-09 14:57'),
 ('김태윤', '홍지은', 2479308, '01-08 02:27'),
 ('이주원', '송지후', 3185166, '01-05 09:05'),
 ('김다은', '최예은', 4149519, '01-20 12:37'),
 ('박서준', '윤지원', 2472899, '01-07 14:36'),
 ('김태윤', '서민재', 809578, '01-12 21:30'),
 ('신소율', '오유진', 4158315, '01-16 13:06'),
 ('김태윤', '신소율', 3630356, '01-23 00:03'),
 ('신도윤', '송지후', 4955042, '01-25 21:42'),
 ('최예은', '서민재', 737213, '01-09 01:54'),
 ('강지호', '최하은', 1315401, '01-10 15:50'),
 ('김태윤', '정하늘', 715035, '01-14 16:50'),
 ('신소율', '신민지', 4804

### 전체 그래프 그리기

In [78]:
# 전체 그래프 그리기
net = Network(height='800px', width='100%', directed=True)

for sender, receiver, amount, time in transactions:
    net.add_node(sender, label=sender)
    net.add_node(receiver, label=receiver)
    net.add_edge(sender, receiver, title=f"{amount}원\n{time}", value=amount)

net.save_graph('full_graph2.html')

In [ ]:
# 부분 그래프 그리기
G = nx.DiGraph()
for sender, receiver, amount, time in transactions:
    G.add_edge(sender, receiver, amount=amount, time=time)

# BFS (너비 우선 탐색)
def find_edges(G, start_node, max_depth):
    visited = set()
    visited.add(start_node) # 시작 노드 방문 추가
    queue = deque()
    queue.append((start_node, 0))
    result_edges = set()

    while queue:
        node, depth = queue.popleft()
        if depth >= max_depth:
            continue
        # 출금 (내가 돈을 보낸 사람)
        for neighbor in G.successors(node): 
            result_edges.add((node, neighbor))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
        # 입금 (나에게 돈을 보낸 사람)
        for neighbor in G.predecessors(node):
            result_edges.add((neighbor, node))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
    return list(result_edges)

start_node = '배수아'
max_depth = 3
sub_edges = find_edges(G, start_node, max_depth)

# pyvis 시각화
net = Network(height='800px', width='100%', directed=True)
added_nodes = set()
for sender, receiver in sub_edges:
    for node in (sender, receiver):
        if node not in added_nodes:
            net.add_node(node, lable=node)
            added_nodes.add(node)
    edge_data = G.get_edge_data(sender, receiver)
    if edge_data: # edge_data가 있으면
        net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])
    else:
        # edge_data가 없을 수도 있으므로 예외 처리
        net.add_edge(sender, receiver)

# 중심 노드 강조
net.get_node(start_node)['color'] = 'red'

# 저장
output_path = 'graph.html'
net.save_graph(output_path)
output_path

'graph.html'

### 커뮤니티 디텍션

In [64]:
# Louvain, Leidan같은 커뮤니티 디텍션 알고리즘들은
# 방향성 없는 그래프를 기본으로 가정
G_undirected = G.to_undirected()

# 커뮤니티 검출
partition = community.best_partition(G_undirected)

# 결과
partition

{'윤지우': 0,
 '김다은': 0,
 '오유진': 3,
 '배수아': 3,
 '신민지': 2,
 '신소율': 5,
 '이주원': 1,
 '홍수아': 1,
 '정민준': 1,
 '신도윤': 0,
 '박서준': 0,
 '최하은': 5,
 '한지호': 5,
 '권현준': 5,
 '민서연': 5,
 '강지호': 0,
 '안현우': 1,
 '정하늘': 2,
 '서지우': 2,
 '서민재': 3,
 '윤지원': 3,
 '홍지은': 3,
 '최다은': 2,
 '최예은': 4,
 '장하준': 4,
 '송지후': 4,
 '홍지호': 0,
 '한민준': 3}

### 시각화에 커뮤니티 반영

In [68]:
from pyvis.network import Network

color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

net = Network(height='800px', width='100%', directed=True)

for node in G.nodes():
    comm_id = partition.get(node, 0) # 커뮤니티 id 추출
    color = color_list[comm_id % len(color_list)]
    net.add_node(node, label=node, color=color)

for sender, receiver in G.edges():
    net.add_edge(sender, receiver)

net.save_graph('community.html')

In [69]:
from pyvis.network import Network

color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

net = Network(height='800px', width='100%', directed=True)

for node in G.nodes():
    comm_id = partition.get(node, 0) # 커뮤니티 id 추출
    if comm_id != 0:
        continue
    color = color_list[comm_id % len(color_list)]
    net.add_node(node, label=node, color=color)

for sender, receiver in G.edges():
    net.add_edge(sender, receiver)

net.save_graph('community0.html')

AssertionError: non existent node '오유진'